<a href="https://colab.research.google.com/github/r-dube/fakejobs/blob/main/fj_glove_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
# Assumes that GLOVE data is available on locally mounted drive
from google.colab import drive
drive.mount('/content/drive')
glove_file='/content/drive/My Drive/Data/glove.6B.50d.txt'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
# Load the modules used
import numpy as np
import pandas as pd
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Input
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant 
from keras.optimizers import Adam
from keras import metrics
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

In [130]:
# NLTK to remove stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [131]:
# For reproducible results
import random as rn
import os
import tensorflow as tf
os.environ['PYTHONHASHSEED'] = '42'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [132]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [133]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [134]:
df = fj_load_df_from_url()
df = fj_txt_only(df)
print('Maximum text length', df['text'].str.len().max())

Loaded dataframe shape (3576, 18)
Not fraudulent 3395 Fraudulent 181
             job_id  telecommuting  ...  has_questions   fraudulent
count   3576.000000    3576.000000  ...    3576.000000  3576.000000
mean    8882.569911       0.043624  ...       0.493568     0.050615
std     5137.458280       0.204286  ...       0.500029     0.219241
min        1.000000       0.000000  ...       0.000000     0.000000
25%     4446.500000       0.000000  ...       0.000000     0.000000
50%     8862.500000       0.000000  ...       0.000000     0.000000
75%    13286.750000       0.000000  ...       1.000000     0.000000
max    17879.000000       1.000000  ...       1.000000     1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                    0
title                     0
location                 70
department             2324
salary_range           2988
company_profile         692
description               0
requirements            549
benefits               1447
telecommuting            

In [135]:
# Utilities to clean text

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r"", string)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [136]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [137]:
stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [138]:
# clean text
df['text'] = df['text'].map(lambda x: remove_URL(x))
df['text'] = df['text'].map(lambda x: remove_html(x))
df['text'] = df['text'].map(lambda x: remove_emoji(x))
df['text'] = df['text'].map(lambda x: remove_punct(x))
df['text'] = df["text"].map(remove_stopwords)

In [139]:
# train-test split
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)

In [140]:
# Max number of words in a sequence
max_length = 250

# embedding size to be created
# This depends on the GLOVE file loaded earlier
embed_dim = 50

tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(train_text)

word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_text)
train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)

test_sequences = tokenizer.texts_to_sequences(test_text)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

print(f"Shape of train {train_padded.shape}")
print(f"Shape of test {test_padded.shape}")

Shape of train (3039, 250)
Shape of test (537, 250)


In [141]:
# get word -> integer mapping
# for the training data
word2idx = tokenizer.word_index
num_tokens = len(word2idx)
print('Found %s unique tokens.' % num_tokens)

Found 58085 unique tokens.


In [142]:
# load in pre-trained GLOVE word vectors
print('Loading word vectors...')
word2vec = {}
with open(glove_file,encoding="utf8") as f:
    # is just a space-separated text file in the format:
    # word vec[0] vec[1] vec[2] ...
    for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
print('Found %s word vectors.' % len(word2vec))

Loading word vectors...
Found 400000 word vectors.


In [143]:
# prepare embedding matrix
print('Filling pre-trained embeddings...')
num_words = num_tokens + 1
embedding_matrix = np.zeros((num_words, embed_dim))
for word, i in word2idx.items():
      embedding_vector = word2vec.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all zeros.
          embedding_matrix[i] = embedding_vector

Filling pre-trained embeddings...


In [144]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed

embedding_layer = Embedding(
  num_words,
  embed_dim,
  weights=[embedding_matrix],
  input_length=max_length,
  trainable=False
)

In [145]:
print('Building model...')

hidden_size = 32

# create an LSTM network with a single LSTM
input_ = Input(shape=(max_length,))
x = embedding_layer(input_)
x = Bidirectional(LSTM(hidden_size, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(input_, output)
model.compile(
  loss='binary_crossentropy',
  optimizer=Adam(lr=0.01),
  metrics=['accuracy', metrics.FalsePositives(), metrics.FalseNegatives()],
)

model.summary()

Building model...
Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 250, 50)           2904300   
_________________________________________________________________
bidirectional_7 (Bidirection (None, 250, 64)           21248     
_________________________________________________________________
global_max_pooling1d_7 (Glob (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 2,925,613
Trainable params: 21,313
Non-trainable params: 2,904,300
_________________________________________________________________


In [146]:
model.fit(train_padded, train_labels, epochs=10)

Epoch 1/10
95/95 [==============================] - 13s 133ms/step - loss: 0.2085 - accuracy: 0.9398 - false_positives_3: 29.0000 - false_negatives_3: 154.0000
Epoch 2/10
95/95 [==============================] - 13s 139ms/step - loss: 0.1574 - accuracy: 0.9487 - false_positives_3: 0.0000e+00 - false_negatives_3: 156.0000
Epoch 3/10
95/95 [==============================] - 13s 139ms/step - loss: 0.1152 - accuracy: 0.9579 - false_positives_3: 2.0000 - false_negatives_3: 126.0000
Epoch 4/10
95/95 [==============================] - 13s 137ms/step - loss: 0.0764 - accuracy: 0.9760 - false_positives_3: 1.0000 - false_negatives_3: 72.0000
Epoch 5/10
95/95 [==============================] - 13s 138ms/step - loss: 0.0559 - accuracy: 0.9809 - false_positives_3: 1.0000 - false_negatives_3: 57.0000
Epoch 6/10
95/95 [==============================] - 13s 138ms/step - loss: 0.0353 - accuracy: 0.9901 - false_positives_3: 1.0000 - false_negatives_3: 29.0000
Epoch 7/10
95/95 [==========================

In [147]:
pred = model.predict(test_padded)
pred = np.around(pred, decimals = 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)
print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))

Accuracy score: 0.9814 F1 score: 0.7826
